# ids projekti, kivaa :---)

tästä alkaa suuri seikkailu. Olkaamme innokaita ja tehokkaita. 

In [118]:
import json
import pandas as pd
import numpy as np
import urllib.request as ur
import requests
from datetime import timedelta, date

In [119]:
def date_range(n):
    date_list = []

    for single_date in (date.today() - timedelta(n+1) for n in range(n)):
        date_list.append(single_date.strftime('%Y%m%d'))

    return(date_list)

In [120]:
def authorize():
    path = 'APIauth.txt'
    temp = open(path,'r').read().splitlines()
    return (temp[0], temp[1])

In [121]:
def fetch_game_scoreboards():
    games_lists = []

    auth = authorize()

    url = 'https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/scoreboard.json?fordate={date}'
    
    DATE_RANGE = 5
    n=0
    for i in date_range(DATE_RANGE):
        if n%50 == 0:
            print(n)
        
        try:
            json = requests.get(url.format(date=i), auth=auth).json()
        except:
            continue
            
        games_lists.append(json['scoreboard']['gameScore'])
        
        n += 1

    return(games_lists)

In [122]:
scoreboard_data = fetch_game_scoreboards()

scoreboard_data_list = np.hstack(scoreboard_data)

0


In [123]:
game_ids = []
for day in scoreboard_data:
    for data in day:
        game_ids.append(data['game']['ID'])

In [124]:
games = {}
auth = authorize()
lineup_url = 'https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/game_startinglineup.json?gameid='
players_json = requests.get('https://api.mysportsfeeds.com/v1.1/pull/nhl/2017-2018-regular/roster_players.json?fordate=20171025', auth=auth).json()

In [126]:
for id in game_ids:
    game = []
    lineup_json = requests.get(lineup_url + id, auth=auth).json()
    
    home_rookies = 0
    away_rookies = 0
    
    home_avg_weight = 0
    home_size_weight = 20
    away_avg_weight = 0
    away_size_weight = 20
    
    home_avg_age = 0
    home_size_age = 20
    away_avg_age = 0
    away_size_age = 20
    
    for lineup_player in lineup_json['gamestartinglineup']['teamLineup'][0]['expected']['starter']:
        for player in players_json['rosterplayers']['playerentry']:
            if (player['player']['ID'] == lineup_player['player']['ID']):
                
                if ('Age' in player['player']):
                    away_avg_age += int(player['player']['Age'])
                else:
                    away_size_age -= 1
                    
                if ('Weight' in player['player']):
                    away_avg_weight += int(player['player']['Weight'])
                else:
                    away_size_weight -= 1
                    
                if (player['player']['IsRookie'] == 'true'):
                    away_rookies += 1
    
    for lineup_player in lineup_json['gamestartinglineup']['teamLineup'][1]['expected']['starter']:
        for player in players_json['rosterplayers']['playerentry']:
            if (player['player']['ID'] == lineup_player['player']['ID']):
                
                if ('Age' in player['player']):
                    home_avg_age += int(player['player']['Age'])
                else:
                    home_size_age -= 1
                    
                if ('Weight' in player['player']):
                    home_avg_weight += int(player['player']['Weight'])
                else:
                    home_size_weight -= 1
                    
                if (player['player']['IsRookie'] == 'true'):
                    home_rookies += 1
                
    game.append(home_rookies)
    game.append(away_rookies)
    game.append(round(home_avg_age/float(home_size_age)))
    game.append(round(away_avg_age/float(away_size_age)))
    game.append(round(home_avg_weight/float(home_size_weight)))
    game.append(round(away_avg_weight/float(away_size_weight)))
    games[id] = game

In [127]:
games # Returns -> gameID:[home_rookies, away_rookies, home_avg_age, away_avg_age, home_avg_weight, away_avg_weight]

{'40883': [1, 1, 26, 25, 193, 200],
 '40884': [1, 0, 27, 28, 200, 197],
 '40885': [0, 1, 28, 27, 196, 198],
 '40886': [0, 1, 27, 27, 195, 196],
 '40887': [0, 2, 28, 27, 201, 199],
 '40888': [0, 0, 28, 28, 201, 203],
 '40889': [3, 1, 26, 26, 196, 199],
 '40890': [1, 2, 25, 27, 198, 203],
 '40891': [0, 2, 28, 25, 208, 193],
 '40892': [3, 1, 25, 27, 197, 197],
 '40893': [0, 1, 28, 28, 195, 200],
 '40894': [0, 1, 28, 27, 198, 203],
 '40896': [1, 2, 27, 27, 198, 203],
 '40897': [1, 0, 27, 28, 200, 203],
 '40898': [0, 3, 28, 25, 201, 197],
 '40899': [1, 1, 27, 25, 196, 200],
 '40900': [1, 0, 26, 29, 193, 206],
 '40901': [1, 0, 26, 30, 199, 198],
 '40902': [2, 0, 25, 27, 193, 195],
 '40903': [0, 2, 28, 27, 196, 203],
 '40904': [1, 2, 28, 27, 194, 199],
 '40905': [0, 0, 28, 28, 197, 195],
 '40906': [1, 0, 28, 28, 200, 196],
 '40907': [2, 0, 25, 28, 195, 208],
 '40908': [1, 1, 27, 27, 198, 197],
 '40909': [1, 0, 27, 28, 203, 195],
 '40910': [1, 1, 25, 26, 198, 199],
 '43364': [0, 0, 29, 28, 200